# Interactive 3D visualization of Fastscape model outputs

Model outputs can be viewed interactively in 3-d rendered scenes using [ipyfastscape](https://github.com/fastscape-lem/ipyfastscape)'s `TopoViz3d`, very much like [Paraview](https://www.paraview.org/) but in Jupyter notebooks.

In [ ]:
import numpy as np
import xsimlab as xs
import xarray as xr

from fastscape.models import basic_model
from ipyfastscape import TopoViz3d

Let's start by setting and running a simple fastscape simulation. Model outputs will be saved along the `time` dimension (clock).

In [ ]:
in_ds = xs.create_setup(
    model=basic_model,
    clocks={
        'clock': np.arange(0, 2e6 + 2e4, 2e4),
        'time': np.arange(0, 2e6 + 2e4, 2e4),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [201, 201],
        'grid__length': [2e5, 2e5],
        'boundary__status': 'fixed_value',
        'uplift__rate': 1e-3,
        'spl': {
            'k_coef': 1e-6,
            'area_exp': 0.6,
            'slope_exp': 1
        },
        'diffusion__diffusivity': 1e-1
    },
    output_vars={
        'topography__elevation': 'time',
        'drainage__area': 'time',
        'spl__chi': 'time'
    }
)

In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=basic_model)

In [ ]:
out_ds

The output dataset `out_ds` can be readily used with `TopoViz3d`, which creates a graphical interface with the 3-d rendered scene as well as convenient controls for some display properties. Here we also explicitly set `time="time"` to enable animation of the 3D scene using some additional controls.

In [ ]:
app = TopoViz3d(out_ds, canvas_height=600, time_dim="time")

app.show()

It is also possible to control the visualization from the notebook code cells. For example, let's show the time step that is the closest to a given time value (you should see the output of the cell here above updated accordingly):

In [ ]:
app.components['timestepper'].go_to_time(1.15e6)